# 波动率
## 定义
**对数收益率方差**的平方根

方差的定义为:
$$s^{2}=\frac{1}{N} \sum_{i=1}^{N}\left(x_{i}-\bar{x}\right)^{2}$$
其中$x_i$为对数收益率，$\bar{x}$为样本的平均收益率，N为样本数量

为了将方差以年化的形式表示，我们需要将原方差乘以年化因子N，也就是一年的交易周期。例如，当我们使用日频数据时，N就是252，因为这是一年中的交易日数量

在金融计算中，常把收益率均值设为0，以减少一个噪声源来增加精度，则方差公式变为：
$$s^{2}=\frac{1}{N} \sum_{i=1}^{N}\left(x_{i}\right)^{2}$$

得到的此方差实际上是一样本方差，还需要用其估计出总体方差，即：
$$ \sigma ^ {2} $ = $ \frac {N}{N-1} $$
$$\sigma^{2}=\frac{1}{N-1} \sum_{i=1}^{N}\left(x_{i}\right)^{2}$$

### close-to-close
$$\sigma = \sqrt{\frac{1}{N-1} \sum_{i=1}^{N}\left(x_{i}\right)^{2}}$$

### Parkinson
$$ \sigma = \sqrt{\frac {1}{4N\ln 2} \sum _ {i=1}^ {N} (ln \frac {h_i}{l_i} )^2}$$

### Garman-Klass
$$\sigma_{G K}=\sqrt{\frac{1}{N} \sum_{i=1}^{N} \frac{1}{2}\left(\ln \frac{h_{i}}{l_{i}}\right)^{2}-\frac{1}{N} \sum_{i=1}^{N}(2 \ln 2-1)\left(\ln \frac{c_{i}}{o_{i}}\right)^{2}}$$

### Garman-Klass-Yang-Zhang
$$\sigma_{GKYZ} = \sqrt{\frac{1}{N} \sum_{i=1}^{N}\left(\ln \frac{o_{i}}{c_{i-1}}\right)^{2}+\frac{1}{N} \sum_{i=1}^{N} \frac{1}{2}\left(\ln \frac{h_{i}}{l_{i}}\right)^{2}-\frac{1}{N} \sum_{i=1}^{N}(2 \ln 2-1)\left(\ln \frac{c_{i}}{o_{i}}\right)^{2}}$$

# 收盘价–收盘价估计量
## 优点：
+ 抽样特性很容易被理解
+ 偏差容易纠正
+ 可以简单地转换为“典型的每日波幅”的公式形式
## 缺点：
+ 没有充分利用已有数据信息
+ 收敛速度很慢。

# Parkinson估计量
## 优点：
+ 使用日内极差来估计波动率很合理，并且相对于基于时间的抽样方法（如收盘价），其提供了更全面的独立信息。
## 缺点：
+ 只适用于几何布朗运动过程的波动率估计，不能处理趋势和跳空。
+ 会系统性地低估波动率。

# Garman-Klass估计量
## 优点：
+ 效率要比收盘价–收盘价估计量高8倍。
+ 充分利用常见的可获取的价格信息。
## 缺点：
+ 偏差甚至比Parkinson估计量还大。

# Yang-Zhang估计量
## 优点：
+ 具有最小估计误差。
+ 能够处理漂移项和价格跳空。
+ 在可用数据的使用上最为有效。
## 缺点：
+ 如果价格过程由跳空主导，其性能会降低到和收盘价–收盘价估计量差不多。

In [1]:
from pandas_datareader import data
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import tushare as ts

In [2]:
start = datetime.datetime(2017,1,1)#获取数据的时间段-起始时间
end = datetime.date.today()#获取数据的时间段-结束时间
df = data.get_data_yahoo('BTC-USD', start=start, end=end)

df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-12-31,963.742981,947.236023,960.627014,963.742981,99135104.0,963.742981
2017-01-01,1003.080017,958.698975,963.658020,998.325012,147775008.0,998.325012
2017-01-02,1031.390015,996.702026,998.617004,1021.750000,222184992.0,1021.750000
2017-01-03,1044.079956,1021.599976,1021.599976,1043.839966,185168000.0,1043.839966
2017-01-04,1159.420044,1044.400024,1044.400024,1154.729980,344945984.0,1154.729980


In [3]:
def cal_vol(df):
   df['log_ret'] = np.log(df['Close']/df['Close'].shift())
   df = df.dropna()
   N = len(df)
   ret_mean = df['log_ret'].mean()
   c2c_vol = np.sqrt(sum((df['log_ret'])**2) / (N-1) )
   parkinson_vol =np.sqrt(sum((np.log(df['High'] / df['Low']))**2) / (4 * N * np.log(2)))
   gk_volatility = np.sqrt(sum((np.log(df['High'] / df['Low'])**2)) / (2 * N) - sum((2 * np.log(2) - 1) * (np.log(df['Close'] / df['Open']))**2) / N)
   gkyz_volatility = np.sqrt(sum(((np.log(df['Open']/df['Close'].shift()).dropna())**2)) / N + sum((np.log(df['High'] / df['Low'])**2)) / (2 * N) - sum((2 * np.log(2) - 1) * (np.log(df['Close'] / df['Open']))**2) / N)

   print(f'daily_vol\nc2c_vol:{c2c_vol}\nparkinson_vol:{parkinson_vol}\nGK_volatility:{gk_volatility}\nGKYZ_volatility:{gkyz_volatility}')
   print('--------------------------------------------------')
   print(f'anualized_vol\nc2c_vol:{c2c_vol * np.sqrt(365)}\nparkinson_vol:{parkinson_vol * np.sqrt(365)}\nGK_volatility:{gk_volatility * np.sqrt(365)}\nGKYZ_volatility:{gkyz_volatility * np.sqrt(365)}')
cal_vol(df)

daily_vol
c2c_vol:0.0418017438531814
parkinson_vol:0.03991469813974803
GK_volatility:0.03918673371530529
GKYZ_volatility:0.03923634021574968
--------------------------------------------------
anualized_vol
c2c_vol:0.7986211949641401
parkinson_vol:0.7625692372298595
GK_volatility:0.7486614964288594
GKYZ_volatility:0.7496092272891324
